# Gateway Auth Okta 統合

AgentCore Identity を使用すると、AgentCore Runtime または AgentCore Gateway ターゲットでエージェントやツールを呼び出すユーザーやアプリケーションの Inbound アクセス（Inbound Auth）を検証できます。また、エージェントから外部サービスや Gateway ターゲットへの安全な Outbound アクセス（Outbound Auth）も提供します。既存の ID プロバイダー（Amazon Cognito など）と統合しながら、独立して動作するエージェントやユーザーに代わって動作するエージェント（OAuth 経由）の権限境界を適用します。

Inbound Auth は、AgentCore Runtime、AgentCore Gateway、または他の環境でホストされているエージェントやツールを呼び出そうとする呼び出し元を検証します。Inbound Auth は IAM（SigV4 認証情報）または OAuth 認可で動作します。

デフォルトでは、Amazon Bedrock AgentCore は IAM 認証情報を使用します。つまり、エージェントへのユーザーリクエストはユーザーの IAM 認証情報で認証されます。このチュートリアルでは Okta IDP を使用した OAuth を使用するため、AgentCore Runtime リソースまたは AgentCore Gateway エンドポイントを設定する際に以下を指定する必要があります：

- OAuth discovery サーバー URL — OpenID Connect discovery URL のパターン ^.+/.well-known/openid-configuration$ に一致する文字列

- 許可される audience — JWT トークンの許可される audience のリスト

- 許可されるクライアント — 許可されるクライアント識別子のリスト

AgentCore CLI を使用する場合、configure コマンドで AgentCore Runtime の認可タイプ（および OAuth discovery サーバー）を指定できます。CreateAgentRuntime オペレーションや Amazon Bedrock AgentCore コンソールも使用できます。Gateway を作成する場合は、CreateGateway オペレーションまたはコンソールを使用します。

ユーザーがエージェントを使用する前に、クライアントアプリケーションはユーザーに OAuth オーソライザーで認証させる必要があります。クライアントはベアラートークンを受け取り、それを呼び出しリクエストでエージェントに渡します。受信時に、エージェントはアクセスを許可する前に認可サーバーでトークンを検証します。

## 概要

このチュートリアルでは、Okta を ID プロバイダーとして使用して Inbound Auth を設定します。1 人のユーザーとアプリクライアントを持つ Okta テナントをセットアップします。Okta アプリクライアントを使用した Inbound Auth を持つ Amazon Bedrock AgentCore Runtime で既存のエージェントをホストする方法を学びます。また、エージェントが対話する Okta を Inbound Auth に使用する Amazon Bedrock AgentCore Gateway もセットアップします。

### チュートリアルアーキテクチャ

<figure>
    <img src="images/16.png">
</figure>

### チュートリアルの詳細

| 情報             | 詳細                                                                            |
|:-----------------|:-------------------------------------------------------------------------------|
| チュートリアルタイプ | 会話型                                                                        |
| エージェントタイプ   | シングル                                                                       |
| エージェントフレームワーク | Strands Agents                                                              |
| LLM モデル         | Anthropic Claude Sonnet 4                                                     |
| チュートリアルコンポーネント | AgentCore Runtime でエージェントをホスト。Strands Agent と Amazon Bedrock Model を使用 |
| チュートリアル分野   | クロスバーティカル                                                              |
| 例の複雑さ         | 簡単                                                                          |
| Inbound Auth      | Okta                                                                          |
| 使用する SDK       | Amazon BedrockAgentCore Python SDK と boto3                                   |


### 主な機能

* Okta を使用した Inbound Auth と Outbound Auth で Amazon Bedrock AgentCore Runtime にエージェントをホスト
* Okta を使用した Inbound Auth で Amazon Bedrock AgentCore Gateway をホスト
* Amazon Bedrock モデルを使用
* Strands Agents を使用

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* 新しい IAM ロール、ポリシー、ユーザーを作成する IAM 権限
* 新しい AgentCore Agent を作成する IAM 権限
* Okta アカウント
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中であること

## Okta の IDP をセットアップする

アプリクライアントと 1 人のテストユーザーを持つ Okta デモテナントをセットアップしましょう。後でこのチュートリアルでデプロイするエージェントを呼び出すための JWT トークンを提供するために Okta を使用します。すでに Okta アカウントをお持ちの場合は、ログインしてください。

https://developer.okta.com/signup/ にアクセスし、「Sign up for Integrator Free Plan」を選択してサインアップします。

1. アカウントにログインします。<br><br>
2. **Directory** を選択し、**People** をクリックして **Add person** をクリックします。
    <figure>
        <img src="images/9.png">
    </figure><br><br>
3. フォームに入力します。
    <ol type="1">
        <li><b>Activation</b> で <b>Activate now</b> を選択します。</li>
        <li><b>I will set password</b> にチェックを入れ、ユーザーのパスワードを設定します。</li>
        <li><b>User must change password on first login</b> のチェックを外します。</li>
        <li><b>Save</b> をクリックします。</li>
    </ol>

    <figure>
        <img src="images/10.png">
    </figure>
    <br><br>
4. **Applications** を選択し、**Create App Integration** をクリックします。
    <figure>
        <img src="images/1.png">
    </figure>
    <br><br>

5. サインイン方法として **OIDC - OpenID Connect** を選択し、アプリケーションタイプとして **Web Application** を選択して **Next** をクリックします。

    <figure>
        <img src="images/2.png">
    </figure><br><br>
    <ol type="a">
        <li>アプリ統合名に <b>Travel Assistant</b> と入力し、<b>Proof of possession</b> はチェックせず、付与タイプとして <b>Authorization Code</b> を選択します。 
        <br><br>
        <figure>
            <img src="images/3.png">
        </figure>
        <br><br>
        </li>
        <li>サインイン URI を更新して <b>http://127.0.0.1:5000/callback</b> と <b>https://bedrock-agentcore.us-west-2.amazonaws.com/identities/oauth2/callback</b> を含めます。サインアウトリダイレクト URI はそのままにします。
        <br><br>
        <figure>
            <img src="images/4.png">
        </figure>
        <br><br>
        </li>
        <li>割り当てで <b>Allow everyone in your organization to access</b> を選択し、<b>Enable immediate access</b> にチェックを入れたままにします。次に <b>Save</b> をクリックします。
        <br><br>
        <figure>
            <img src="images/5.png">
        </figure>
        <br><br>
        </li>
        <li>後で使用するために <b>Client ID</b> と <b>Secret</b> をコピーします。</li>
        <br><br>
        <figure>
            <img src="images/6.png">
        </figure>
        <br><br>
    </ol><br>

6. 左側のメニューで **Security** を選択し、**API** をクリックして、認可サーバーの名前をクリックします。
    <figure>
        <img src="images/17.png">
    </figure><br><br>
    <ol type="a">
        <li><b>Audience</b> をコピーして後で使用するために保存します。</li>
            <ul>
                <li><b>注意</b>：この例ではデフォルトの <b>Audience</b> が変更されています。他のアプリに影響を与えないように、audience を変更する予定がある場合は新しい認可サーバーを追加することをお勧めします。</li><br>
            </ul>
        <figure>
            <img src="images/7.png">
        </figure>
        <br><br>
        <li><b>Scopes</b> をクリックし、<b>Add Scope</b> をクリックします。</li><br>
        <figure>
            <img src="images/43.png">
        </figure>
        <br><br>
        <li>名前に <b>okta.myAccount.read</b> を使用し、<b>Display Phrase</b> と <b>Description</b> を入力します。</li>
        <li><b>User Consent</b> を <b>implicit</b> に設定します。</li>
        <li><b>Block services</b>、<b>Default scope</b>、<b>Metadata</b> はデフォルトのままにします。</li>
        <li><b>Save</b> をクリックします。
        <figure>
            <img src="images/44.png">
        </figure><br>
        <li><b>Claims</b> をクリックし、以下の <b>client_id</b> と <b>scope</b> クレームを追加します。</li><br>
        <figure>
            <img src="images/8.png">
        </figure><br>
        <li><b>Access Policies</b> をクリックし、<b>Add New Access Policy</b> をクリックします</li><br>
        <figure>
            <img src="images/39.png">
        </figure><br>
        <li><b>Name</b>、<b>Description</b> を入力し、<b>Create Policy</b> をクリックします</li><br>
        <figure>
            <img src="images/40.png">
        </figure><br>
        <li><b>Add rule</b> をクリックします</li><br>
        <figure>
            <img src="images/41.png">
        </figure><br>
        <li>ルールに <b>Rule Name</b> を付けて <b>Create Rule</b> をクリックします</li><br>
        <figure>
            <img src="images/42.png">
        </figure><br>
    </ol>




## API Gateway の作成

この CloudFormation テンプレートは JWT 認証を備えたサーバーレス API を作成します：
* Okta からのトークンを検証する JWT オーソライザーを備えた Amazon API Gateway
* 有効な JWT 認証が必要な **GET /travel-plans** の保護されたエンドポイント
* **GET /travel-plans** エンドポイントの呼び出しで実行される AWS Lambda 関数

AgentCore Gateway は、MCP リクエストを基盤となる API Gateway エンドポイントへの HTTP 呼び出しに変換するプロトコルアダプターとして機能することで、この Amazon API Gateway を MCP（Model Context Protocol）サーバーに変換します。

以下のコードセルを実行して、CloudFormation テンプレートを template.yaml としてローカルファイルシステムに保存します。

In [ ]:
%%writefile template.yaml
AWSTemplateFormatVersion: '2010-09-09'
Parameters:
  JwtIssuerUrl:
    Type: String
    Description: The URL of the JWT issuer (e.g., Cognito user pool URL).
    MinLength: 10 # Optional: You can add constraints like minimum length
    MaxLength: 200 # Optional: Maximum length
    # You can also add a Default value if you want

  JwtAudienceList:
    Type: CommaDelimitedList # <-- Using CommaDelimitedList for multiple audiences
    Description: A comma-separated list of expected audience(s) for the JWT (e.g.,
      "my-api-audience-1,my-api-audience-2").
    # Optional: You can add Default or other constraints if needed
    # Default: "my-default-audience"

Resources:
  MyHttpApi:
    Type: AWS::ApiGatewayV2::Api
    Properties:
      Name: MyHttpApi
      ProtocolType: HTTP

  MyJwtAuthorizer:
    Type: AWS::ApiGatewayV2::Authorizer
    Properties:
      ApiId: !Ref MyHttpApi
      AuthorizerType: JWT
      IdentitySource:
        - $request.header.Authorization
      JwtConfiguration:
        Audience: !Ref JwtAudienceList
        Issuer: !Ref JwtIssuerUrl
      Name: MyJwtAuthorizer

  MyLambdaFunction:
    Type: AWS::Lambda::Function
    Properties:
      FunctionName: MyLambdaFunction
      Handler: lambda_function.lambda_handler
      Runtime: python3.12
      Code:
        ZipFile: |
          exports.handler = async (event) => {
              // In non-proxy integration, the Lambda function receives the mapped input
              // from API Gateway, NOT the full HTTP request.
              console.log('Received event:', JSON.stringify(event, null, 2));

              const name = event.name || "World";
              const message = `Hello, ${name}! (from non-proxy integration)`;

              // In non-proxy integration, you can return a simple string, object, etc.
              // API Gateway then formats this into a proper HTTP response using mapping templates.
              return { "message": message }; // Example of a simple JSON response
          };
      MemorySize: 128
      Timeout: 30
      Role: !GetAtt MyLambdaExecutionRole.Arn

  MyLambdaExecutionRole:
    Type: AWS::IAM::Role
    Properties:
      AssumeRolePolicyDocument:
        Version: '2012-10-17'
        Statement:
          - Effect: Allow
            Principal:
              Service:
                - lambda.amazonaws.com
            Action:
              - sts:AssumeRole
      Policies:
        - PolicyName: MyLambdaPolicy
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: Allow
                Action:
                  - logs:CreateLogGroup
                  - logs:CreateLogStream
                  - logs:PutLogEvents
                Resource: arn:aws:logs:*:*:*

  MyApiIntegration:
    Type: AWS::ApiGatewayV2::Integration
    Properties:
      PayloadFormatVersion: "2.0"
      ApiId: !Ref MyHttpApi
      IntegrationType: AWS_PROXY
      IntegrationUri: !Join
        - ''
        - - 'arn:'
          - !Ref 'AWS::Partition'
          - ':apigateway:'
          - !Ref 'AWS::Region'
          - ':lambda:path/2015-03-31/functions/'
          - !GetAtt MyLambdaFunction.Arn
          - /invocations
      IntegrationMethod: POST # Lambda invocations are typically POST
      # You'll also need to define mapping templates with AWS integration type
      # as shown in the IntegrationRequest and IntegrationResponse sections

  MyApiRoute:
    Type: AWS::ApiGatewayV2::Route
    Properties:
      ApiId: !Ref MyHttpApi
      RouteKey: GET /travel-plans # Changed to POST to match IntegrationMethod
      Target: !Join
        - /
        - - integrations
          - !Ref MyApiIntegration
      AuthorizationType: JWT
      AuthorizerId: !Ref MyJwtAuthorizer

1. us-west-2 リージョンで CloudFormation にアクセスし、Create stack をクリックします。
2. **With new resources(standard)** を選択します。
<figure>
    <img src="images/20.png">
</figure>

3. 前提条件として **Choose an existing template** を選択します。
4. テンプレートの指定で **Upload a template file** を選択し、前のステップで保存した **template.yaml** ファイルを選択します。
5. **Next** をクリックします。

<figure>
    <img src="images/21.png">
</figure>

6. **stack name** を入力します（例：test-deployment-stack）
7. JwtAudienceList パラメータに **Audience** を入力します。
8. JwtIssueURL に **Issuer URL** を入力します（例：https://{yoursubdomain}.okta.com/oauth2/default）。

<figure>
    <img src="images/22.png">
</figure>

9. **I acknowledge that AWS CloudFormation might create IAM resources** にチェックを入れます。
10. **Next** をクリックします。
<figure>
    <img src="images/32.png">
</figure>

11. **Submit** をクリックします。

<figure>
    <img src="images/49.png">
</figure>

## Lambda 関数の更新

1. **Lambda** > **Functions** にアクセスし、**MyLambdaFunction** を**クリック**します。
<figure>
    <img src="images/23.png">
</figure>

2. 以下のソースコードを **Code** セクションにコピーします。
3. <b>index.js</b> ファイルを <b>lambda_function.py</b> にリネームします
4. **Deploy** をクリックします。
<figure>
    <img src="images/24.png">
</figure>

5. **API Gateway** > **Integrations** にアクセスし、**Manage integrations** をクリックします。
<figure>
    <img src="images/56.png">
</figure>

6. **Integration details** の下にある **Edit** をクリックします。
<figure>
    <img src="images/57.png">
</figure>

7. 最新の **Lambda function** ARN に更新し、**Save** をクリックします。
<figure>
    <img src="images/58.png">
</figure>

AWS Lambda コード：

In [ ]:
import json
from datetime import datetime, timedelta
import random
import uuid
# Mock data storage (in production, this would be a database)
MOCK_TRAVEL_PLANS = [
    {
        "id": "plan-001",
        "user_id": "user-123",
        "email": "john.doe@example.com",
        "destination": "Paris, France",
        "departure_date": "2024-03-15",
        "return_date": "2024-03-22",
        "accommodation": "Hotel Le Marais",
        "activities": ["Eiffel Tower", "Louvre Museum", "Seine River Cruise"],
        "budget": 2500.00,
        "status": "confirmed"
    },
    {
        "id": "plan-002",
        "user_id": "user-123",
        "email": "john.doe@example.com",
        "destination": "Tokyo, Japan",
        "departure_date": "2024-05-10",
        "return_date": "2024-05-20",
        "accommodation": "Tokyo Grand Hotel",
        "activities": ["Mount Fuji", "Sensoji Temple", "Shibuya Crossing"],
        "budget": 3500.00,
        "status": "planned"
    },
    {
        "id": "plan-003",
        "user_id": "user-456",
        "email": "jane.smith@example.com",
        "destination": "New York, USA",
        "departure_date": "2024-04-01",
        "return_date": "2024-04-07",
        "accommodation": "Manhattan Plaza Hotel",
        "activities": ["Statue of Liberty", "Central Park", "Broadway Show"],
        "budget": 2000.00,
        "status": "confirmed"
    },
    {
        "id": "plan-004",
        "user_id": "user-456",
        "email": "jane.smith@example.com",
        "destination": "Barcelona, Spain",
        "departure_date": "2024-06-15",
        "return_date": "2024-06-25",
        "accommodation": "Barcelona Beach Resort",
        "activities": ["Sagrada Familia", "Park Güell", "Las Ramblas"],
        "budget": 2800.00,
        "status": "planned"
    },
    {
        "id": "plan-005",
        "user_id": "user-789",
        "email": "bob.wilson@example.com",
        "destination": "Sydney, Australia",
        "departure_date": "2024-07-20",
        "return_date": "2024-08-03",
        "accommodation": "Sydney Harbour Hotel",
        "activities": ["Opera House", "Harbour Bridge", "Bondi Beach"],
        "budget": 4500.00,
        "status": "tentative"
    }
]
def lambda_handler(event, context):
    """
    Main Lambda handler for travel plans API
    """
    query_parameters = event.get('queryStringParameters', {})
    try:
        # Route based on HTTP method and path
        return get_travel_plans(query_parameters)
    except Exception as e:
        return create_response(500, {
            'error': 'Internal Server Error',
            'message': str(e)
        })
def create_response(status_code, body):
    """
    Create API Gateway Lambda response
    """
    return {
        'statusCode': status_code,
        'body': json.dumps(body)
    }
def get_travel_plans(query_params):
    """
    Get travel plans by user_id or email
    Query parameters:
    - user_id: Filter by user ID
    - email: Filter by email address
    """
    user_id = query_params.get('user_id') if query_params else None
    email = query_params.get('email') if query_params else None
    
    # Validate that at least one parameter is provided
    if not user_id and not email:
        return create_response(400, {
            'error': 'Either user_id or email must be provided',
            'message': 'Please provide user_id or email as query parameter'
        })
    
    # Filter travel plans based on the provided parameter
    filtered_plans = []
    
    for plan in MOCK_TRAVEL_PLANS:
        if user_id and plan['user_id'] == user_id:
            filtered_plans.append(plan)
        elif email and plan['email'].lower() == email.lower():
            filtered_plans.append(plan)
    
    # Sort by departure date (most recent first)
    filtered_plans.sort(key=lambda x: x.get('departure_date', ''), reverse=True)
    
    # Return response
    if filtered_plans:
        return create_response(200, {
            'success': True,
            'count': len(filtered_plans),
            'travel_plans': filtered_plans,
            'filter': {
                'user_id': user_id,
                'email': email
            }
        })
    else:
        return create_response(404, {
            'success': False,
            'message': 'No travel plans found for the specified user',
            'filter': {
                'user_id': user_id,
                'email': email
            },
            'travel_plans': []
        })


## Amazon Bedrock AgentCore Gateway の作成

以下のコードは **demo_openapi.yaml** ファイルをハードドライブに書き込みます。このファイルは、前のステップで作成した API Gateway へのリクエストをプロキシする Amazon Bedrock AgentCore Gateway の作成に使用されます。
<ol type="1">
<li><b>API Gateway</b> > <b>APIs</b> > <b>Stages</b> にアクセスし、<b>Create</b> をクリックします。</li>
<figure>
    <img src="images/50.png">
</figure>
<li>ステージの <b>Name</b> を <b>default</b> に設定します。</li>
<li><b>Enable automatic deployment</b> にチェックを入れます。</li>
<li><b>Create</b> をクリックします。</li>
<figure>
    <img src="images/51.png">
</figure>
<li><b>Default endpoint</b> をメモします。</li>
<figure>
    <img src="images/52.png">
</figure>
<li>以下のコードセルをクリックして <b>demo_openapi.yaml</b> ファイルをハードドライブに保存します。</li>
    <ol type="a">
            <li><b>demo_openapi.yaml</b> を開き、<b>server URL</b> の <b>{yoursubdomain}</b> を前のステップの <b>Invoke URL</b> に合わせて編集します。</li>
            <li><b>authorization URL</b> と <b>token URL</b> の <b>{yoursubdomain}</b> を Okta テナントに合わせて編集し（Okta テナントにサインインする際に使用する URL に記載されています）、ファイルを保存します。</li>
    </ol>
</ol>

In [ ]:
%%writefile demo_openapi.yaml
openapi: 3.0.0
info:
  title: Travel Plans API
  description: API for retrieving user travel plans with secure authentication
  version: 1.0.0
  contact:
    name: Travel Plans Developer
    email: developer@example.com
  license:
    name: Apache 2.0
    url: http://www.apache.org/licenses/LICENSE-2.0.html

servers:
  - url: https://{yoursubdomain}.execute-api.us-west-2.amazonaws.com/default
    description: Production server

paths:
  /travel-plans:
    get:
      summary: Retrieve Travel Plans
      description: Fetch travel plans by user ID or email
      operationId: getTravelPlans
      security:
        - OAuth2:
          - read:travel-plans
        - ApiKeyAuth: []
      parameters:
        - in: query
          name: user_id
          schema:
            type: string
          required: false
          description: Unique identifier of the user
          example: "user-123"
        
        - in: query
          name: email
          schema:
            type: string
            format: email
          required: false
          description: Email address of the user
          example: "john.doe@example.com"
      
      responses:
        '200':
          description: Successful retrieval of travel plans
          content:
            application/json:
              schema:
                type: object
                properties:
                  success:
                    type: boolean
                  count:
                    type: integer
                  travel_plans:
                    type: array
                    items:
                      $ref: '#/components/schemas/TravelPlan'
                  filter:
                    type: object
                    properties:
                      user_id:
                        type: string
                      email:
                        type: string
        
        '400':
          description: Bad Request - Missing query parameters
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        
        '401':
          description: Unauthorized - Invalid or missing authentication
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        
        '403':
          description: Forbidden - Insufficient permissions
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        
        '404':
          description: No travel plans found
          content:
            application/json:
              schema:
                type: object
                properties:
                  success:
                    type: boolean
                  message:
                    type: string
                  filter:
                    type: object
                    properties:
                      user_id:
                        type: string
                      email:
                        type: string
                  travel_plans:
                    type: array
                    items: {}
        
        '500':
          description: Internal Server Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'

components:
  securitySchemes:
    OAuth2:
      type: oauth2
      flows:
        authorizationCode:
          authorizationUrl: https://{yoursubdomain}.okta.com/oauth2/default/v1/authorize
          tokenUrl: https://{yoursubdomain}.okta.com/oauth2/default/v1/token
          scopes:
            read:travel-plans: Read access to travel plans
            write:travel-plans: Write access to travel plans
            delete:travel-plans: Delete access to travel plans
  
    ApiKeyAuth:
      type: apiKey
      in: header
      name: X-API-Key

    BearerAuth:
      type: http
      scheme: bearer
      bearerFormat: JWT

  schemas:
    TravelPlan:
      type: object
      required:
        - id
        - user_id
        - email
        - destination
        - departure_date
        - return_date
        - status
      properties:
        id:
          type: string
          description: Unique identifier for the travel plan
          example: "plan-001"
        user_id:
          type: string
          description: Unique identifier of the user
          example: "user-123"
        email:
          type: string
          format: email
          description: Users email address
          example: "john.doe@example.com"
        destination:
          type: string
          description: Travel destination
          example: "Paris, France"
        departure_date:
          type: string
          format: date
          description: Date of departure
          example: "2024-03-15"
        return_date:
          type: string
          format: date
          description: Date of return
          example: "2024-03-22"
        accommodation:
          type: string
          description: Name of accommodation
          example: "Hotel Le Marais"
        activities:
          type: array
          items:
            type: string
          description: List of planned activities
          example: ["Eiffel Tower", "Louvre Museum"]
        budget:
          type: number
          format: float
          description: Estimated travel budget
          example: 2500.00
        status:
          type: string
          description: Status of the travel plan
          enum:
            - confirmed
            - planned
            - tentative
          example: "confirmed"

    ErrorResponse:
      type: object
      properties:
        error:
          type: string
        message:
          type: string
        error_code:
          type: string
        timestamp:
          type: string
          format: date-time

    OAuthToken:
      type: object
      properties:
        access_token:
          type: string
          description: JWT access token
        token_type:
          type: string
          enum:
            - Bearer
        expires_in:
          type: integer
          description: Token expiration time in seconds
        refresh_token:
          type: string
          description: Token to obtain a new access token

tags:
  - name: Travel Plans
    description: Operations related to travel plan retrieval
  - name: Authentication
    description: OAuth2 and API Key authentication methods

x-security-definitions:
  - name: OAuth2
    description: >
      OAuth 2.0 Authentication:
      - Authorization Code Flow
  - name: API Key
    description: >
      API Key authentication for service-to-service communication

x-rate-limiting:
  limit: 100
  period: 1 minute
  
x-error-handling:
  generic-errors:
    - 400: Bad Request
    - 401: Unauthorized
    - 403: Forbidden
    - 404: Not Found
    - 500: Internal Server Error

2. **demo_openapi.yaml** を S3 バケットにアップロードします。アップロード先のバケットは自由に選択できます。
<figure>
    <img src="images/25.png">
</figure>

3. **Amazon Bedrock AgentCore** にアクセスします。
4. 左側のメニューで **Identity** を選択します
5. **Add OAuth client / API key** をクリックし、**Add Oauth client** をクリックします。
<figure>
    <img src="images/26.png">
</figure>

6. Provider で **Custom provider** を選択します。
7. **Discovery URL** を選択し、**Client ID**、**Client secret**、**Discovery URL**（https://{yoursubdomain}.okta.com/oauth2/default/.well-known/openid-configuration）を入力し、後で使用するためにリソースプロバイダーの名前を保存します。
8. **Add OAuth Client** をクリックします。
<figure>
    <img src="images/27.png">
</figure>


9. 左側のメニューで **Gateways** を選択し、**Create Gateway** をクリックします。
<figure>
    <img src="images/28.png">
</figure>

10. **Gateway name** を設定するか、デフォルト名を使用します。
<figure>
    <img src="images/29.png">
</figure>

11. **Inbound Auth configurations** で **Use existing Identity provider configurations** を選択します。
12. **Discovery URL**（https://{yoursubdomain}.okta.com/oauth2/default/.well-known/openid-configuration）、**Allowed audiences**、**Allowed clients**（Okta テナントの audience と Client ID）を入力します。
<figure>
    <img src="images/30.png">
</figure>

13. Target:{target name} で、ターゲットタイプとして **REST API** を選択します。
14. REST API タイプで **OpenAPI schema** を選択します。
15. OpenAPI schema で **Define with an S3 resource** を選択します。
16. **ステップ 2** の openapi ドキュメントの場所にアクセスして選択します。
17. Outbound Auth configurations で **OAuth client** を選択します。
18. OAuth client で、**ステップ 3〜5** で作成した Amazon Bedrock AgentCore Identity を選択します。
<figure>
    <img src="images/31.png">
</figure>

19. Scopes で <b>okta.myAccount.read</b> を追加します。
20. **Save** をクリックします。
<figure>
    <img src="images/33.png">
</figure>



 
## AgentCore Runtime へのデプロイ用にエージェントを準備する

このコードは、Strands フレームワークと Bedrock AgentCore SDK を使用して **Travel Assistant チャットボット** を定義します。

顧客 ID またはメールを使用して Amazon Bedrock AgentCore Gateway 経由で既存の旅行計画を取得できるトラベルアシスタントエージェントをセットアップします。@requires_access_token() デコレーターは、アクセストークンを取得してデコレートされた関数に注入するプロセスを自動的に管理することで OAuth 2.0 認証を処理します。前のステップで作成した Outbound OAuth クライアントを使用して Okta との認証フローを設定し、必要なスコープ（"okta.myAccount.read"）をリクエストし、認証が必要な場合はユーザーがアプリケーションを認可するために訪問する認可 URL をコンソールに出力します。ユーザーが OAuth フローを完了すると、デコレーターは取得したアクセストークンを need_token_3LO_async 関数にパラメータとして自動的に注入し、繰り返しの認証リクエストを避けるためにトークンをトークンボールトに保存します。

1. 以下のセルを実行して、エージェントコードを **my_agent_mcp.py** としてローカルファイルシステムに保存します。
2. <b>Amazon AgentCore Gateway</b> > <b>Gateways</b> にアクセスし、Gateway（例：gateway-quick-start-234a1）をクリックします。
<figure>
    <img src="images/53.png">
</figure>

3. ファイルを見つけて、gateway_url の **{yoursubdomain}** を **Gateway resource URL** に合わせて置き換えます。
<figure>
    <img src="images/38.png">
</figure>

4. provider_name の **{yourprovidername}** を Outbound OAuth クライアントの名前（例：resource-provider-oauth-client-1wbak）に置き換えます。
5. ファイルを保存します。

In [ ]:
%%writefile my_agent_mcp.py
import json
import requests
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent, tool
from strands.tools.mcp import MCPClient
from strands_tools import calculator, current_time

# Import the AgentCore SDK
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token

WELCOME_MESSAGE = """
Welcome to the Travel Assistant! How can I help you today?
"""

SYSTEM_PROMPT = """
You are an helpful travel support assistant.
When provided with a customer email, gather all necessary info and prepare the response.
When asked about existing travel plans, look for it and customize the summary based on the prompt.
Don't mention the customer ID in your reply.
"""

# Global token storage
okta_access_token = None

def create_streamable_http_transport(mcp_url: str, access_token: str):
       return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})

# Create an AgentCore app
app = BedrockAgentCoreApp()

async def agent_task(user_message: str) -> None:

    global okta_access_token
    okta_access_token = await need_token_3LO_async(access_token='')
    response = ''

    gateway_url = "https://{yoursubdomain}.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp"
    mcp_client = MCPClient(lambda: create_streamable_http_transport(gateway_url , okta_access_token))

    with mcp_client:
        #tools = await get_full_tools_list(mcp_client)
        #print(f"Found the following tools: {[tool.tool_name for tool in tools]}")
    
        agent = Agent(tools=mcp_client.list_tools_sync())
        response = agent(user_message)
        print(response)
    
    return response.message['content'][0]['text']

# Injects Okta Access Token
@requires_access_token(# Uses the same credential provider name created above
    provider_name= "{yourprovidername}",
    # Requires Okta OAuth2 scope to access MCP Server
    scopes= ["okta.myAccount.read"],
    # Sets to OAuth 2.0 Authorization Code flow
    auth_flow= "USER_FEDERATION",
    # Prints authorization URL to console
    on_auth_url= lambda x: print("\nPlease copy and paste this URL in your browser:\n" + x),
    # If false, caches obtained access token
    force_authentication= False,) 
async def need_token_3LO_async(*, access_token: str) -> str:
    """Handle OAuth authentication flow."""
    global okta_access_token
    okta_access_token = access_token
    return access_token


# Specify the entry point function invoking the agent
@app.entrypoint
async def invoke(payload):
    """Handler for agent invocation"""
    user_message = payload.get(
        "prompt", "No prompt found in input, please guide customer to create a json payload with prompt key"
    )

    result = await agent_task(user_message)
    return result

if __name__ == "__main__":
    app.run()

このコードを実行するには、Strands Agents モジュールを Python 環境にインストールする必要があります。

依存関係をインストールするには、仮想環境を作成してアクティブ化します：

In [ ]:
!python -m venv .venv 
!source .venv/bin/activate

Strands Agents モジュール、AgentCore SDK、AgentCore starter toolkit を依存関係ファイルに追加し、**requirements.txt** として保存します：

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit


次に、仮想環境にすべての要件をインストールします：

In [ ]:
%pip install -r requirements.txt

## エージェントを AgentCore Runtime にデプロイする
`CreateAgentRuntime` オペレーションは包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントとどのように通信するかを制御するためのプロトコル設定（HTTP、MCP）と認可メカニズムも設定できます。

注意：運用のベストプラクティスは、CI/CD パイプラインと IaC を使用してコードをコンテナとしてパッケージ化し、ECR にプッシュすることです

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore runtime にデプロイできます。

### AgentCore Runtime デプロイ用にエージェントを設定する
次に、starter toolkit を使用して、エントリーポイント、先ほど作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するように starter kit を設定します。

設定ステップでは、アプリケーションコードに基づいて docker ファイルが生成されます

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

discovery_url = input("Enter your discovery URL: ")

client_id = input("Enter your client ID: ")

audience = input("Enter your audience: ")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="my_agent_mcp.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_inbound_identity_okta",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id],
            "allowedAudience": [audience]
        }
    }
)
response

### AgentCore 設定を確認する

In [ ]:
!cat .bedrock_agentcore.yaml

#### エージェントを AgentCore Runtime に起動する

docker ファイルができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<figure>
    <img src="images/14.png">
</figure>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

#### AgentCore Runtime のステータスを確認する

AgentCore Runtime をデプロイしたので、デプロイステータスを確認しましょう。

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### 実行ロールに OAuth トークンの読み取り権限を付与する

1. <b>Amazon Bedrock AgentCore<b> にアクセスします。
2. 左側のメニューで <b>Agent Runtime</b> をクリックします。
3. エージェント（例：strands_agent_inbound_identity_okta）をクリックします。
<figure>
    <img src="images/45.png">
</figure>

4. 最新バージョンをクリックします。
<figure>
    <img src="images/46.png">
</figure>

5. <b>IAM service role</b> をクリックします。
<figure>
    <img src="images/47.png">
</figure>

6. <b>Permissions policies</b> の下にある <b>Policy name</b> をクリックします。
<figure>
    <img src="images/48.png">
</figure>

7. エージェントにトークンボールトへのアクセスを許可するために、以下のステートメントを追加します。

		{
			"Sid": "GetResourceOauth2Token",
			"Effect": "Allow",
			"Action": [
				"bedrock-agentcore:GetResourceOauth2Token",
				"secretsmanager:GetSecretValue"
			],
			"Resource": "*"
		}

8. <b>Next</b> をクリックします。
<figure>
    <img src="images/59.png">
</figure>

### エージェントに Amazon Bedrock モデルアクセスを付与する
1. <b>Amazon Bedrock</b> > <b>Model access</b> にアクセスし、<b>Modify model access</b> をクリックします。
<figure>
    <img src="images/54.png">
</figure>

2. <b>Claude Sonnet 4</b> にチェックを入れて <b>Next</b> をクリックします（注意：strands が使用するデフォルトモデルは変更される可能性があります）。
<figure>
    <img src="images/55.png">
</figure>

#### 認可なしで AgentCore Runtime を呼び出す

最後に、ペイロードを使用して AgentCore Runtime を呼び出すことができます。以下のセルを実行すると、**"AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeAgentRuntime operation: Agent is configured for a different authorization token type"** というエラーが表示されます。

<figure>
    <img src="images/15.png">
</figure>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "What are the travel plans for customer 123?"})
invoke_response

#### 認可付きで AgentCore Runtime を呼び出す

正しい認可トークンタイプでエージェントを呼び出しましょう。この場合、Okta アクセストークンになります。

以下のコマンドは、次のステップの OAuth サーバーに必要な **Flask** フレームワークから **Requests** ライブラリをインストールします。

In [ ]:
%pip install flask requests

以下のコードは、アクセストークンを取得できるようにサーバーを立ち上げます。起動したら、リンクをクリックするか http://127.0.0.1:5000 にアクセスして OAuth フローを完了します。アクセストークンを取得したら、起動した方法と同じ方法でサーバーを停止します。

以下のコードは：
1. OAuth 認証を処理するための小さな Web サーバーを Flask で作成します。
2. ユーザーが ```/login``` にアクセスすると、ログインのために Okta にリダイレクトされます。
3. ログイン後、サーバーは /callback ルート経由で認証コードを受け取ります。
4. コードは次にトークンエンドポイントへの POST リクエストでアクセストークンと交換されます。
5. 成功すると、アクセストークンはセッションに保存され、コンソールに出力されます。

**注意**：認可 URL とトークン URL は discovery URL（https://{yoursubdomain}.okta.com/oauth2/default/.well-known/openid-configuration）から見つけることができます。

In [ ]:
import os
import requests
import secrets
from flask import Flask, redirect, request, session, url_for

app = Flask(__name__)
app.secret_key = os.urandom(24)

# === Configuration ===
CLIENT_ID = input("Enter your Client ID: ")
CLIENT_SECRET = input("Enter your Client Secret: ")
AUTHORIZATION_BASE_URL = input("Enter your authorization URL: ")
TOKEN_URL = input("Enter your token URL: ")
REDIRECT_URI = "http://127.0.0.1:5000/callback"
SCOPE = "openid email"  # Adjust according to the provider

# === Step 1: Redirect to Authorization Server ===
@app.route("/")
def home():
    return '<a href="/login">Login with OAuth</a>'

@app.route("/login")
def login():
    state = secrets.token_urlsafe(16)
    session["oauth_state"] = state

    auth_url = (
        f"{AUTHORIZATION_BASE_URL}?response_type=code&client_id={CLIENT_ID}"
        f"&redirect_uri={REDIRECT_URI}&scope={SCOPE}&state={state}"
    )
    return redirect(auth_url)

# === Step 2: Handle Callback and Exchange Code for Token ===
@app.route("/callback")
def callback():
    error = request.args.get("error")
    if error:
        return f"Error: {error}"

    code = request.args.get("code")
    if not code:
        return "No code found"

    token_data = {
        "grant_type": "authorization_code",
        "code": code,
        "redirect_uri": REDIRECT_URI,
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
    }

    token_response = requests.post(TOKEN_URL, data=token_data)
    token_json = token_response.json()

    if "access_token" in token_json:
        session["access_token"] = token_json["access_token"]
        print("Access Token: " + token_json["access_token"])

        return "Login successful! Access token in logs."
    else:
        return f"Failed to get token: {token_json}"

if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5000)

アクセストークンをコピーし、次のセクションのコードで求められたときに入力します。

In [ ]:
bearer_token = input("Enter your bearer token: ")
invoke_response = agentcore_runtime.invoke(
    {"prompt": "What flights does customer with user id user-123 have scheduled?"}, 
    bearer_token=bearer_token
)
invoke_response

1. AWS コンソールで Amazon Bedrock AgentCore にアクセスし、Build and Deploy の下にある Agent Runtime をクリックします
2. エージェント（例：my_agent_mcp）をクリックします。
<figure>
    <img src="images/36.png">
</figure>

3. **Runtime ID** をコピーします。
<figure>
    <img src="images/34.png">
</figure>

4. ターミナルで以下のコマンドを実行して、エージェントからログを取得します。**{myagentruntimeid}** をエージェントの **Runtime ID** に置き換えてください。

```aws logs tail /aws/bedrock-agentcore/runtimes/{myagentruntimeid} --follow```

5. 認可リクエスト URL をキャプチャし、ブラウザに貼り付けて OAuth ハンドシェイクを完了します。

<figure>
    <img src="images/35.png">
</figure>

## おめでとうございます！